## Import packages

In [1]:
#image analysis
import skimage.io
import imageio
import alienlab.plot
from alienlab.improcessing import normalize, grey_to_rgb, make_binary
import alienlab.segment
from alienlab.fo import FramesOperator
import alienlab.io


#interactive widget packages
from ipywidgets import interact, interactive, fixed, interact_manual
from tkinter.filedialog import askopenfilename


import time
import os
import numpy as np
import matplotlib.pyplot as plt



%matplotlib qt

## Import video file

#### USER ACTION:

#### **Drag and drop a video file (.tif format) in the file section**

In [2]:
file_path = "ENTER NAME OF THE FILE YOU DROPPED IN THE FILE SECTION HERE"
#file_path = askopenfilename(title = 'Select a video file') # pops up a window to select your file
# uncomment this line if you use this jupyter notebook locally
#

In [3]:
show = True #option to output intermediary images in the segmentation process

# Import video file in HQ and select ROI

direc = os.path.split(file_path)[0]

# Initialize plotting tools
g = alienlab.plot.ShowFigure()
g.figsize = (15,7)
g.save_folder = "images"
g.date = False
p = alienlab.plot.PlotFigure()
p.figsize = (15,7)
p.save_folder = "images"
p.date = False

## Pre-process the video

In [4]:
# read the stacked frame. dim = NxHxW (N images in the video, Heigt, Width)

frames_full = skimage.io.imread(file_path)

#frames_full = np.stack([frames_full[:,:,1]]*10, 0) 
#uncomment this line if you have a single RGB image. The [:,:,1] stands for selection of the green channel

FO = FramesOperator(frames_full)
im = normalize(FO.frames[0], 0, 1)
im = grey_to_rgb(im)*255

#y, x = alienlab.io.select_roi(np.uint8(im)) #select area of interest
#FO.x = x
#FO.y = y
#FO.crop() #crop image

start_time = time.time()
FO.compute_stats() #compute various statistical values on the frames and the pixels
FO.normalize(0, 1)
print("--- Computed frames statistics in %04f seconds ---" % (time.time() - start_time))

#FO.global_stats: each array has size N, number of frames and represents the stats of each frame
#FO.frames_stats: each array has size FO.x, FO.y and is an image representing the N frames stats overlayed

if show:
    p.title = 'statistics'
    p.xlabel = 'frame number'
    p.ylabel = 'amplitude'
    p.label_list = ['max', 'min', 'mean', 'std']
    fig = p.plotting(np.asarray(FO.inds), [FO.global_stats['max'], 
                        FO.global_stats['min'], 
                        FO.global_stats['mean']])
    p.save_name = 'frames_stats'
    p.saving(fig)

''' IMAGE SEGMENTATION '''

# selection of the frames with high dynamics that will be used for the image segmentation process.
# Let M be the highest value taken by a pixel in all the frames of the video. The frame F is kept for processing only if at
# least one pixel in the frame F has a value above 0.8*M. 
FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.8)


--- Computed frames statistics in 1.513992 seconds ---


### Show reference image

In [5]:
plt.figure(figsize = (5, 5))
plt.imshow(FO.frames[FO.selected_inds].sum(axis = 0), cmap = 'gray')
plt.axis('off')


(-0.5, 2751.5, 2207.5, -0.5)

## Image segmentation

In [6]:
def segment_image(contrast, autolevel, dist_max, dist_seg, disk_size, max_contrast, interact = True, showit = show):
    
    start_time = time.time()
    FO.selected_inds = FO.select_frames(FO.global_stats['max'], FO.global_stats['max'].max()*0.8) # Select only images with high intensity to increase contrast and lower computation time

    #apply contrast filter to all frames
    frames_contrast = FO.apply(skimage.filters.rank.enhance_contrast,  selem = skimage.morphology.disk(contrast))
    #apply autolevel filter to all frames
    frames_autolevel = FO.apply(skimage.filters.rank.autolevel, selem = skimage.morphology.disk(autolevel))
    #sum the contrast images to get a reference grey-level contrast image
    frame_contrast = np.sum(frames_contrast, axis = 0)
    #sum the autolevel images to get a reference grey-level autolevel image
    frame_autolevel = np.sum(frames_autolevel, axis = 0)
    #obtain contrast mask from reference contrast image
    mask_contrast = make_binary(frame_contrast, soft_hard = 1)
    #otbain autolevel mask from reference autolevel image
    mask_autolevel =  make_binary(frame_autolevel, soft_hard = 1)
    #intersection of contrast aud autolevel masks
    mask_intersect = mask_contrast * mask_autolevel
    #clean the masks with a binary opening
    mask_intersect = skimage.morphology.binary_opening(mask_intersect, selem = skimage.morphology.disk(disk_size))
    #reference image of altitude for the watershed
    auto_contrast = normalize(mask_intersect * frame_autolevel)
    print("--- Computed binary mask in %04f seconds ---" % (time.time() - start_time))

    g.cmap = "inferno"
    if showit:
        g.figsize = (40,15)
        g.title_list =  'contrast', 'contrast threshold', 'mask intersect','autolevel', 'autolevel threshold','segmentation image'
        g.col_num = 3
        fig = g.multi([frame_contrast, mask_contrast, mask_intersect, 
                       frame_autolevel, mask_autolevel,  auto_contrast])
        g.save_name = 'Segmentation reference'
        g.saving(fig)

    start_time = time.time()
    ref = auto_contrast
    mask = mask_intersect
    #locate the local maxima
    local_maxi = alienlab.segment.local_maxima(auto_contrast, max_contrast, g,
                                                     ref_distance = dist_max, mask = mask, show = showit)
    #perform watershed segmentation
    watershed_im_mask = alienlab.segment.watershed(ref, mask, local_maxi,
                                                         g, ref_distance = dist_seg, show = False)
    segmented = watershed_im_mask
    print("--- Computed segmentation in %04f seconds ---" % (time.time() - start_time))

    if showit:
        alienlab.segment.show_segmentation(FO, segmented, g)
        
    if interact == False:
        return watershed_im_mask, FO


In [ ]:
interact(segment_image, contrast = (1, 4), autolevel = (1, 10), dist_max = False, dist_seg = False, disk_size = (1, 4), max_contrast = (1,10), soft_hard = (0.1, 1.9))

In [7]:
mask, FO = segment_image(contrast = 2, autolevel = 5, dist_max = False, dist_seg=False, disk_size = 2, max_contrast = 3, interact = False, showit= False)

--- Computed binary mask in 15.022845 seconds ---
--- Computed segmentation in 4.045800 seconds ---


In [10]:
g.cmap = "flag"
fig = g.multi(mask)